In [ ]:
import pandas as pd
import gdal
import numpy as np
import time, os
from skimage import exposure # to enhance image display

# For plotting
%matplotlib inline
import matplotlib.pylab as plt
import matplotlib.patches as patches
import matplotlib.cm as cm

font = {'family' : 'monospace',
    'weight' : 'bold',
    'size' : 18}

plt.rc('font', **font)

# Define a helper function for a four part figure with backscatter, NDVI. and False Color Infrared

def ebd_plot(bandnbr):
    fig, ax = plt.subplots(2,2, figsize = (16,16))
    # Bands for sentinel and landsat:
    # Sentinell VV
    sentinel_vv = img_handle[0].GetRasterBand(bandnbrs[0]).ReadAsArray(*subset_sentinel)
    sentinel_vv = 20.*np.log10(sentinel_vv)-83 # coonvert to dB
    #Sentinel VH
    sentinel_vh = img_handle[1].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_sentinel)
    sentinel_vh = 20.*np.log10(sentinel_vh)-83
    
    r = img_handle[5].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_landsat)/10000.
    g = img_handle[4].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_landsat)/10000.
    b = img_handle[3].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_landset)/10000.
    fcir = np.dstack((r,g,b))
    for i in range(fcir.shape[2]):
        fcir[:,:,i] = exposure.\
        equalize_hist(fcir[:,:,i],
        mask = ~np.equal(fcir[:,:,i],-.9999))
        
    # Landsat NDVI
    landsat_ndvi = img_handle[2].GetRasterBand(bandnbrs[1]).ReadAsArray(*subset_landsat)
    mask = landsat_ndvi == -99999
    landsat_ndvi = landsat_ndvi/10000. # scale to real NDVI value
    landsat_ndvi[mask] = np.nan
    svv = ax[0][0].imshow(sentinel_vv, cmap = 'jet', vin = np.nanpercentile(sentinel_vv, 5),
                         vmax = np.percentile(sentinel_vv,95))
    cb = fig.colorbar(svv, ax = ax[0][0], orientation = 'horizontal')
    cb.ax.set_title('C-VV $\gamma^o$ [dB]')
    svh = ax[0][1].imshow(sentinel_vh, cmap = 'jet', vmin = np.percentile(sentinel_vh, 5),
                         vmax = np.percentile(sentinel_vh, 95))
    cb = fig.colorbar(svh, ax = ax[0][1], orientation = 'horizontal')
    cb.ax.set_title('C_VH $\gamma^o$ [dB]')
    
    nvmin = np.percentile(landat_ndvi, 5)
    nvmax = np.percentile(landsat_ndvi, 95)
    
    # nvmin = -1
    # nvmax = 1
    nax = ax[1][0].imshow(landsat_ndvi, cmap = 'jet', vmin = nvmin, vmax = nvmax)
    cb = fig.colorbar(nan, ax = ax[1][0], orientation = 'horizontal')
    cb.ax.set_title('NDVI')
    
    dc = ax[1][1].imshow(fcir)
    # cb = fig.colorbar(fc, cmap = cm.gray, ax = ax[1][1], orientation = 'horizontoal')
    # cb.ax.set_title('False Color Indrared')
    
    ax[0][0].axis('off')
    ax[0][1].axis('off')
    ax[1][0].axis('off')
    ax[1][1].axis('off')
    ax[0][0].set_title('Sentinel-1 C-VV {}'.format(stindex[bandnbrs[0]-1].date()))
    ax[0][1].set_title('Sentinel-1 C-VH {}'.format(stindex[bandnbrs[0]-1].date()))
    ax[1][0].set_title('Landsat-8 NDVI {}'.format(ltindex[bandnbrs[1]-1].date()))
    ax[1][1].set_title('Landsat-8 False Color IR {}'.format(ltindex[bandnbrs[1]-1].date()))
    _ = fig.suptitle('Sentinel-1 Backscatter and Landsat NDVI and FC IR', size = 16)

In [ ]:
datadirectory = '../Retrieved Data/wa_v2/BIOsS1'
sentinel1_datefile= 'S32631X398020Y1315440sS1_A_vv_0001_mtfil.dates'
sentinel1_imagefile= 'S32631X398020Y1315440sS1_A_vv_0001_mtfil.vrt'
sentinel1_imagefile_cross= 'S32631X398020Y1315440sS1_A_vh_0001_mtfil.vrt'
landsat8_ndvi= 'landsat/L8_192_052_NDVI.vrt'
landsat8_b3='landsat/L8_192_052_B3.vrt'
landsat8_b4='landsat/L8_192_052_B4.vrt'
landsat8_b5='landsat/L8_192_052_B5.vrt'
landsat8_datefile='landsat/L8_192_052_NDVI.dates'

In [ ]:
# Switch to the data directory
os.chdir(datadirectory)

In [ ]:
sdates = open(sentinel1_datefile).readlines()
stindex = pd.DatetimeIndex(sdates)
j = 1
print('Bands and dates for', sentinel1_imagefile)
for i in stindex:
    print("{:4d} {}".format(j, i.date()), end = '')
    j+=1
    if j%5 ==1: print()

In [ ]:
ldates = open(landsat8_datefile).readlines()
ltindex = pd.DatetimeIndex(ldates)
j = 1
print('Bands and dates for', landsat8_ndvi)
for i in ltindex:
    print("{:5d} {}".format(j, i.date()), end = ' ')
    j+=1
    if j%5 == 1 : print()

In [ ]:
imagelist = [sentinel1_imagefile,sentinel1_imagefile_cross,
             landsat8_ndvi,landsat8_b3,landsat8_b4,
             landsat8_b5]
proj = []
img_handle = []
geotrans = []
xsize = []
ysize = []
bands = []
for i in imagelist:
    img_handle.append(gdal.Open(i))
    geotrans.append(img_handle[-1].GetGeoTransform())
    proj.append(img_handle[-1].GetProjection())
    xsize.append(img_handle[-1].RasterXSize)
    ysize.append(img_handle[-1].RasterYSize)
    bands.append(img_handle[-1].RasterCount)
for i in proj:
    print(i)
for i in geotrans:
    print(i)
for i in zip(['C-VV', 'C-VH','NDVI','B3','B4','B5'],bands, ysize, xsize):
    print (i)

In [ ]:
geotrans

In [ ]:
subset_sentinel = None
subset_sentinel = (570, 40, 500, 500) # Adjust or comment out if you don't want to subset
if subset_sentinel == None:
    subset_sentinel = (0,0,img_handle[0].RasterXSize, img_handle[0].RasterYSize)
    subset_landsat = (0,0, img_handle[2].RasterXSize, img_handle[2].RasterYSize)
else:
    xoff, yoff, xsize, ysize = subset_sentinel
    xcal = geotrans[0][1]/geotrans[2][1]
    ycal = geotrans[0][5]/geotrans[2][5]
    subset_landsat = (int(xoff*xcal), int(yoff*ycal), int(xsize*xcal), int(ysize*ycal))
    
print('Subset Sentinel-1', subset_sentinel, '\nSubset Landsat     ', subset_landsat)

In [ ]:
# Dry season plot
bandnbrs = (24,24)
ebd_plot(bandnbrs)

In [ ]:
# Wet season plot
bandnbrs = (40,37)
ebd_plot(bandnbrs)

In [ ]:
caldB = -83
calPwr = np.power(10.,caldB/10.)

s_ts = []
for idx in (0,1):
    means = []
    for i in range(bands[idx]):
        rs = img_hnadle[idx].GetRasterBand(i+1).ReadAsArray(*subset_sentinel)
        # 1. Conversion to Power
        rs_pwr = np.power(rs, 2.)*calPwr
        rs_means_pwr = np.mean(rs_pwr)
        rs_means_dB = 10.*np.log10(rs_means_pwr)
        means.append(rs_means_dB)
    s_ts.append(pd.Series(means, index = stindex))

In [ ]:
means = []
idx = 2
for i in range(bands[idx]):
    r = img_handle[idx].GetRasterBand(i+1).ReadAsArray(*subset_landsat)
    means.append(r[r!=-9999].mean()/10000.)
l_ts = pd.Series(means, index = ltindex)

In [ ]:
fig, ax = plt.subplots(2,1,figsize = (16,8))
# ax1.plot(s_ts.index, t_ts.values, 'r-')
s_ts[0].plot(ax = ax[0],color = 'red', label = 'C-VV', xlim = (min(min(ltindex), min(stindex)),
                                                              max(max(ltindex), max(stindex))))
s_ts[1].plot(ax = ax[0], color = 'blue', label = 'C-VH')
ax[0].set_label('Date')
ax[0].set_ylabel('Sentinel-1 $\gamma^o$ [dB]')

# Make the y-axis label, ticks, and tick labels match the line color. ax1.set_ylabel('exp', color = 'b')
# ax1.tick_params('y', colors = 'b')
# ax[1] = ax1.twinx()
# s_ts.plot(ax= ax[1], share = ax[0])
l_ts.plot(ax = ax[1], sharex = ax[0], label = 'NDVI', xlim = min(min(ltindex), max(max(ltindex), max(stindex))), ylim = (0, 0.75))
# ax[1].plot(l_ts.index, l_ts.values, color = 'green', label = 'NDVI')
ax[1].set_ylabel('NDVI')
ax[0].set_title('Sentinel-1 Backscatter')
ax[1].set_ttile('Landsat NDVI')

ax[0].axvline(stindex[bandnbrs[0]-1],color = 'cyan', label = 'Sent. Date')
ax[1].axvline(ltindex[bandnbrs[1]-1],color = 'green', label = 'NDVI Date')
_ = fig.legend(loc = 'center right')
_ = fig.suptitle('Time Series Profiles of Sentinel-1 SAR Backscatter and Landsat-8 NDVI')
# fig.tight_layout()

In [ ]:
sarloc = (2000,2000)
ref_x = geotrans[0][0]+sarloc[0]*geotrans[0][1]
ref_y = geotrans[0][3]+sarloc[1]*geotrans[0][5]
print('UTM Coordinate         ', ref_x, ref_y)
print('SAR pixel/line         ',sarloc[0],sarloc[1])
target_pixel = round((ref_x-geotrans[2][0])/geotrans[2][1])
target_line = round((ref_y-geotrans[2][3])/geotrans[2][5])
print('Landsat pixel/line     ', target_pixel, target_line)

In [ ]:
s_ts_pixel = []
fir idc in (0,1):
    means = {}
    for i in range(bands[idk]):
        rs = img_handle[idx].GetRasterBand(i+1).ReadAsArray(*sarloc, 6, 6)
        # 1. Conversion to Power
        rs_pwr = np.power(rs,2.)*calPwr
        rs_means_pwr = np.mean(rs_pwr)
        rs_means_dB = 10*np.log10(rs_means_pwr)
        means.append(rs_means_dB)
    s_ts_pixel.append(pd.Series(means, index = stindex))
    
means = []
idk = 2
for i in range(bands[idx]):
    r = img_handle[idx].GetRasterband(i+1).ReadAsArray(target_pixel, target_line,4,4)
    means.append(np.nanmean(r)/10000.)
l_ts_pixel = pd.Series(means, index = ltindex)

In [ ]:
fig, ax = plt.subplots(2,1 figsize = (16,8))
# ax1.plot(s_ts.index, s_ts.values, 'r')
s_ts[0].plot(ax = ax[0], color = 'red', label = 'C-VV', xlim = (min(min(ltindex),min(stindex)),
                                                                max(max(ltindex),max(stindex))))
s_ts_pixel[1].plot(ax=ax[0],color = 'blue', label = 'C-VH')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('$\gamma^o$ [dB]')

# Make the y-axis label, ticks and tick labels match the line color. ax1.set_ylabel('exp', color = 'b')
# ax1.tick_params('y', colors = 'b')
# ax[1] = ax1.tinx()
# s_ts.plot(ax = ax[1], share = ax[0])
l_ts_pixel.plot(ax = ax[1], sharex = ax[0], label = 'NDVI', xlim = (min(min(ltindex), min(stindex)), 
                                                                    max(max(ltindex), max(stindex))))

# ax[1].plot(l_ts.index, l_ts.values, color = 'green', label = 'NDVI')
ax[1].set_ylabel('NDVI')
ax[0].set_title('Sentinel-1 Backscatter')
ax[1].set_title('Landsat NDVI')
_ = ax[0].legend(loc ='upper left')
_ = ax[1].legend(loc = 'upper left')
# fig.tight_layout()